In [28]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 
from sklearn.model_selection import train_test_split

In [3]:
tokenizer = Tokenizer()

data_saadi = open('poetry_saadi.txt').read()
data_rumi = open('poetry_rumi.txt').read()

corpus_saadi = data_saadi.lower().split("\n")
corpus_rumi = data_rumi.lower().split("\n")

In [11]:
corpus = corpus_saadi+ corpus_rumi
corpus = list(filter(None,corpus))

In [12]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'نفخ': 2803, 'بی\u200cصورتی': 6315, 'همی\u200cگذرانیم': 4154, 'عرصه': 3566, 'طوع': 4169, 'همزبان': 6415, 'طویلست': 4289, 'افکند': 1529, 'بجوشد': 4982, 'کانچ': 7377, 'بصر': 914, 'مهر': 268, 'چابک': 6740, 'عهده': 3342, 'می\u200cگداخت': 4943, 'مجتبی': 4800, 'بودست': 831, 'بسوی': 2675, 'گفتیم': 1323, 'ترست': 3055, 'موم': 1947, 'تعبیه': 6453, 'کای': 428, 'موجود': 1200, 'ازکیها': 4410, 'زجر': 2286, 'پرده': 638, 'نبندد': 4559, 'گواهی': 3290, 'مدینه': 6654, 'رغبت': 3751, 'اصل\u200cجو': 5552, 'خوانده': 6757, 'سرکه': 3004, 'قماش': 6084, 'لابه': 1780, 'بهاست': 3896, 'نقره': 1257, 'بی\u200cتو': 5496, 'سحارت': 3722, 'حیران': 648, 'سیمینست': 4444, 'آسان': 1740, 'همتی': 4062, 'اقصای': 6925, 'سوخته': 1957, 'درمانست': 3164, 'تن': 154, 'رهگذار': 4574, 'خلد': 1753, 'می\u200cزنی': 1597, 'مانند': 855, 'بخفت': 4832, 'می\u200cنگنجد': 2902, 'ساق': 3738, 'پرواز': 1549, 'ناوک': 2046, 'منا': 2859, 'خونش': 4988, 'ببینندت': 7333, 'فهم': 544, 'مردی': 1296, 'هونا': 7327, 'روش': 2003, 'لقمانست': 7496, 'سرند': 4553, 

In [25]:
input_sequences = []
labels = []
for line in corpus_saadi:
    token_list = tokenizer.texts_to_sequences([line])[0]
    input_sequences.append(token_list)
    labels.append(0)
for line in corpus_rumi:
    token_list = tokenizer.texts_to_sequences([line])[0]
    input_sequences.append(token_list)
    labels.append(1)
    
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, ys = input_sequences, np.array(labels)
# ys = tf.keras.utils.to_categorical(labels, num_classes=2)

In [32]:
X_train,X_test,Y_train,Y_test = train_test_split(xs,ys,test_size=0.20,random_state=40)

In [44]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(150)))
# model.add(Conv1D(150,1,activation='relu'))
# model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
adam = Adam(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(X_train, Y_train, epochs=100, verbose=1)
#print model.summary()
print(model)

Train on 5004 samples
Epoch 1/100
5004/5004 [==============================] - 13s 3ms/sample - loss: 0.3827 - accuracy: 0.8301
Epoch 2/100
5004/5004 [==============================] - 8s 2ms/sample - loss: 0.0899 - accuracy: 0.9586
Epoch 3/100
5004/5004 [==============================] - 9s 2ms/sample - loss: 0.0395 - accuracy: 0.9796
Epoch 4/100
5004/5004 [==============================] - 8s 2ms/sample - loss: 0.0283 - accuracy: 0.9826
Epoch 5/100
5004/5004 [==============================] - 9s 2ms/sample - loss: 0.0282 - accuracy: 0.9806
Epoch 6/100
5004/5004 [==============================] - 9s 2ms/sample - loss: 0.0246 - accuracy: 0.9812
Epoch 7/100
5004/5004 [==============================] - 9s 2ms/sample - loss: 0.0246 - accuracy: 0.9812
Epoch 8/100
5004/5004 [==============================] - 9s 2ms/sample - loss: 0.0236 - accuracy: 0.9830
Epoch 9/100
5004/5004 [==============================] - 8s 2ms/sample - loss: 0.0228 - accuracy: 0.9828
Epoch 10/100
5004/5004 [========

In [45]:
score = model.evaluate(X_test,Y_test, verbose=0)
print('Accurace : {}'.format(score[1]))

Accurace : 0.8570287823677063


In [46]:
seed_text = "ای ضیاء الحق حسام الدین بیار"
token_list = tokenizer.texts_to_sequences([seed_text])[0]
token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
predicted = model.predict_classes(token_list, verbose=0)
print(predicted)

[[1]]
